In [1]:
# --- gplume_pasig_hourly_demo.py -------------------------------------------
# Trial Gaussian plume over Pasig for one hour (forward + optional inverse).

import os, math
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from shapely.geometry import Point

# ---------------- PATHS ----------------
CSV_PATH   = r"C:\Users\HP\Desktop\SpatialCARE\Hourly\pasig_hourly_corrected.csv"  # hourly stations CSV
PASIG_SHP  = r"C:\Users\HP\Desktop\SpatialCARE\Pasig\Pasig.shp"
OUT_DIR    = r"C:\Users\HP\Desktop\SpatialCARE\Hourly\HourlyOutputs\gplume"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------------- USER CONFIG ----------------
# Pick a test hour that exists in your CSV; or set to None to auto-pick the first
TEST_HOUR = None  # e.g., "2025-07-01 18:00"

# Columns in your CSV (auto-detect candidates are fine; hardwire if you prefer)
PM_CANDS    = ["pm25","PM25","PM_25","PM2_5","PM2.5"]
LAT_CANDS   = ["latitude","lat","LAT","y","Y","Lat"]
LON_CANDS   = ["longitude","lon","LONG","x","X","Lon"]
SITE_CANDS  = ["location_name","stations","station","Station","STATION","site","Site"]
DT_CANDS    = ["datetime","date_time","DateTime","DATETIME","timestamp","Timestamp"]
DATE_CANDS  = ["date","Date","DATE"]
TIME_CANDS  = ["time","Time","TIME","hour","Hour","HOUR","HH"]

# Sources (edit to your suspected emitters)
# Each source: dict(name, lat, lon, Q_gps, H_m)
SOURCES = [
    # Example: a nominal source near Pasig center; adjust coords/Q as needed
    {"name": "SrcA", "lat": 14.56, "lon": 121.08, "Q_gps": 1.0, "H_m": 10.0},
    # Add more sources as needed...
]

# Wind for the selected hour (if you don’t have hourly wind yet)
# Wind direction (deg FROM which it blows; meteorological), speed m/s
DEFAULT_WD_DEG = 90.0   # wind from the EAST? (90° means from EAST to WEST); adjust as needed
DEFAULT_WS     = 2.0

# Stability class (Pasquill-Gifford): A..F. Neutral urban → 'D' is a reasonable first try.
STABILITY = 'D'

# Grid resolution (meters) for the concentration surface
GRID_RES_M = 150

# Flip this to True to try the PyPI 'gplume' package if installed
USE_PYPI_GPLUME = False   # requires: pip install gplume

# ---------------- UTIL ----------------
def pick(cols, cands):
    for c in cands:
        if c in cols: return c
    return None

def to_datetime(df):
    dtc = pick(df.columns, DT_CANDS)
    if dtc: return pd.to_datetime(df[dtc], errors="coerce")
    d = pick(df.columns, DATE_CANDS); t = pick(df.columns, TIME_CANDS)
    if d and t: return pd.to_datetime(df[d].astype(str)+" "+df[t].astype(str), errors="coerce")
    if d: return pd.to_datetime(df[d], errors="coerce")
    raise SystemExit("No datetime/date/time columns found.")

def wd_to_unit_xy(wd_deg_met):
    """
    Convert meteorological wind direction (degrees FROM) to unit vector (ux, uy) blowing TO.
    0° = from north, 90° = from east.
    """
    theta = np.deg2rad((270.0 - wd_deg_met) % 360.0)  # convert to math heading (0 along +x)
    return np.cos(theta), np.sin(theta)

def rotate_coords(x, y, ux, uy):
    """
    Rotate coordinates so x' is downwind axis.
    Given wind unit vector (ux,uy) blowing TO, rotate (x,y) into plume coords (x_down, y_cross).
    """
    # Basis vectors: e_x' = (ux, uy); e_y' = (-uy, ux)
    x_down =  x*ux + y*uy
    y_cross = -x*uy + y*ux
    return x_down, y_cross

# --- Pasquill-Gifford sigmas (simple rural/neutral-ish parameterization).
def sigmas_xy(x_m, stability='D'):
    x = np.asarray(x_m).clip(min=1.0)  # avoid zero
    s = stability.upper()
    # Rough, widely used forms (Turner workbook-esque); demo-level only.
    if s == 'A':
        sig_y = 0.22*x*(1+0.0001*x)**-0.5
        sig_z = 0.20*x
    elif s == 'B':
        sig_y = 0.16*x*(1+0.0001*x)**-0.5
        sig_z = 0.12*x
    elif s == 'C':
        sig_y = 0.11*x*(1+0.0001*x)**-0.5
        sig_z = 0.08*x*(1+0.0002*x)**-0.5
    elif s == 'D':
        sig_y = 0.08*x*(1+0.0001*x)**-0.5
        sig_z = 0.06*x*(1+0.0015*x)**-0.5
    elif s == 'E':
        sig_y = 0.06*x*(1+0.0001*x)**-0.5
        sig_z = 0.03*x*(1+0.0003*x)**-1.0
    else: # 'F'
        sig_y = 0.04*x*(1+0.0001*x)**-0.5
        sig_z = 0.016*x*(1+0.0003*x)**-1.0
    return sig_y, sig_z

def gaussian_plume_ground(Q_gps, u_ms, x_m, y_m, H_m, stability='D'):
    """
    Ground-level concentration at z=0 with ground reflection (simple).
    Units: Q[g/s], u[m/s] -> C[g/m^3]
    """
    x = np.asarray(x_m)
    y = np.asarray(y_m)
    C = np.zeros_like(x, dtype=float)
    mask = x > 0  # only downwind
    if not np.any(mask) or u_ms <= 0: return C
    sig_y, sig_z = sigmas_xy(x[mask], stability)
    pref = Q_gps / (2.0 * math.pi * u_ms * sig_y * sig_z)
    cross = np.exp(-(y[mask]**2) / (2.0*sig_y**2))
    refl = 2.0 * np.exp(-(H_m**2) / (2.0*sig_z**2))  # z=0, image term doubles
    C[mask] = pref * cross * refl
    return C

# ---------------- LOAD DATA ----------------
if not os.path.exists(CSV_PATH): raise SystemExit(f"Missing CSV: {CSV_PATH}")
df_raw = pd.read_csv(CSV_PATH)
pm_col  = pick(df_raw.columns, PM_CANDS) or "pm25"
lat_col = pick(df_raw.columns, LAT_CANDS)
lon_col = pick(df_raw.columns, LON_CANDS)
site_col= pick(df_raw.columns, SITE_CANDS) or "site"
if site_col not in df_raw.columns: df_raw[site_col] = "Station"

dt = to_datetime(df_raw)
df = pd.DataFrame({
    "site": df_raw[site_col].astype(str),
    "datetime": pd.to_datetime(dt, errors="coerce"),
    "lat": pd.to_numeric(df_raw[lat_col], errors="coerce"),
    "lon": pd.to_numeric(df_raw[lon_col], errors="coerce"),
    "pm25": pd.to_numeric(df_raw[pm_col], errors="coerce").clip(lower=0)
}).dropna(subset=["datetime","lat","lon","pm25"])

# Pick the hour to model
df["datetime"] = df["datetime"].dt.floor("h")
hour_list = np.sort(df["datetime"].unique())
if TEST_HOUR is None:
    if len(hour_list) == 0: raise SystemExit("No hourly rows found.")
    test_time = hour_list[0]
else:
    test_time = pd.to_datetime(TEST_HOUR)
    if test_time not in set(hour_list):
        raise SystemExit(f"Requested hour {TEST_HOUR} not in data.")

obs = df[df["datetime"] == test_time].copy()
if obs.empty: raise SystemExit("No observations at selected hour.")

# ---------------- GEOMETRY ----------------
g_pasig = gpd.read_file(PASIG_SHP)
if g_pasig.crs is None: g_pasig = g_pasig.set_crs(4326)
g_pasig = g_pasig.to_crs(32651)  # UTM 51N (meters)

g_obs = gpd.GeoDataFrame(
    obs.copy(),
    geometry=gpd.points_from_xy(obs["lon"], obs["lat"]),
    crs=4326
).to_crs(32651)

# Source geometry in projected coords
src_df = pd.DataFrame(SOURCES)
g_src = gpd.GeoDataFrame(
    src_df.copy(),
    geometry=gpd.points_from_xy(src_df["lon"], src_df["lat"]),
    crs=4326
).to_crs(32651)

# Fixed map extent
xmin, ymin, xmax, ymax = g_pasig.total_bounds
pad = 300
xmin, ymin, xmax, ymax = xmin-pad, ymin-pad, xmax+pad, ymax+pad

# Receptor grid (inside Pasig bbox)
xs = np.arange(xmin, xmax, GRID_RES_M)
ys = np.arange(ymin, ymax, GRID_RES_M)
GX, GY = np.meshgrid(xs, ys)
grid_pts = np.c_[GX.ravel(), GY.ravel()]
g_grid = gpd.GeoDataFrame(geometry=gpd.points_from_xy(grid_pts[:,0], grid_pts[:,1]), crs=32651)
# Mask to Pasig polygon
mask_in = g_grid.within(g_pasig.unary_union)
grid_pts = grid_pts[mask_in.values]
GXm = grid_pts[:,0].reshape((-1,1))  # we'll rebuild as flat then reshape later
GYm = grid_pts[:,1].reshape((-1,1))

# ---------------- WIND ----------------
# For now, use a single (ws, wd) for the hour. Plug your hourly winds if available.
ws = float(DEFAULT_WS)
wd = float(DEFAULT_WD_DEG)
ux, uy = wd_to_unit_xy(wd)  # unit vector blowing TO

# ---------------- FORWARD MODEL ----------------
# Sum contributions of all sources.
C_sum = np.zeros((grid_pts.shape[0],), dtype=float)

for i, s in g_src.iterrows():
    # Translate grid to source-centered coords
    dx = grid_pts[:,0] - s.geometry.x
    dy = grid_pts[:,1] - s.geometry.y
    # Rotate into downwind/crosswind
    x_down, y_cross = rotate_coords(dx, dy, ux, uy)
    # Compute concentration (g/m^3)
    Ci = gaussian_plume_ground(
        Q_gps=float(src_df.loc[i, "Q_gps"]),
        u_ms=ws,
        x_m=x_down,
        y_m=y_cross,
        H_m=float(src_df.loc[i, "H_m"]),
        stability=STABILITY
    )
    C_sum += Ci

# Convert to µg/m^3 for plotting
C_ugm3 = C_sum * 1e6

# ---------------- OPTIONAL: quick inverse fit (estimate Q for given sources)
DO_INVERSE = True
Q_est = None
if DO_INVERSE and len(g_src) >= 1 and len(g_obs) >= 1:
    # Build design matrix G: concentration at receptors for unit emission (Q=1 g/s) from each source
    R = np.c_[g_obs.geometry.x.values, g_obs.geometry.y.values]
    S = np.c_[g_src.geometry.x.values, g_src.geometry.y.values]
    G = []
    for j, s in enumerate(S):
        dxR = R[:,0] - s[0]
        dyR = R[:,1] - s[1]
        xR, yR = rotate_coords(dxR, dyR, ux, uy)
        C_unit = gaussian_plume_ground(1.0, ws, xR, yR, float(src_df.loc[j,"H_m"]), STABILITY)  # g/m^3 per g/s
        G.append(C_unit)
    G = np.vstack(G).T  # shape (n_receptors, n_sources)
    y_obs = (g_obs["pm25"].values) * 1e-6  # convert µg/m^3 -> g/m^3
    # Solve least squares for nonnegative Q (clip negatives)
    Q_hat, _, _, _ = np.linalg.lstsq(G, y_obs, rcond=None)
    Q_hat = np.clip(Q_hat, 0, None)
    Q_est = Q_hat
    # Recompute grid with estimated Qs
    C_inv = np.zeros_like(C_sum)
    for j, s in enumerate(S):
        dx = grid_pts[:,0] - s[0]
        dy = grid_pts[:,1] - s[1]
        xg, yg = rotate_coords(dx, dy, ux, uy)
        C_inv += gaussian_plume_ground(Q_hat[j], ws, xg, yg, float(src_df.loc[j,"H_m"]), STABILITY)
    C_inv_ugm3 = C_inv * 1e6
else:
    C_inv_ugm3 = None

# ---------------- PLOT ----------------
title_wind = f"Wind: {ws:.1f} m/s, {wd:.0f}° (from)"
stamp = pd.to_datetime(test_time).strftime("%Y-%m-%d %H:00")

def plot_surface(C_flat_ugm3, label, fname):
    # Reconstruct 2D array on the masked grid (sparse plot)
    g_mask = mask_in.values
    # Make a full canvas at bbox grid, fill with NaN then place masked cells
    nx = len(xs); ny = len(ys)
    Z = np.full((len(ys), len(xs)), np.nan)
    # Map indices
    # Build index arrays for all grid cells
    all_pts = np.c_[np.repeat(xs, len(ys)), np.tile(ys, len(xs))]
    # But we already computed masked order; easiest is to rasterize via GeoDataFrame
    # For speed, just scatter plot instead of full raster:
    fig, ax = plt.subplots(figsize=(7,7), dpi=150)
    ax.set_aspect("equal")
    g_pasig.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1.0, zorder=2)
    sc = ax.scatter(grid_pts[:,0], grid_pts[:,1], c=C_flat_ugm3, s=35, cmap="inferno",
                    norm=LogNorm(vmin=max(1e-3, np.nanmin(C_flat_ugm3[C_flat_ugm3>0])), vmax=np.nanmax(C_flat_ugm3)),
                    edgecolors="none", zorder=1)
    # Sources
    g_src.plot(ax=ax, color="cyan", markersize=45, edgecolor="black", linewidth=0.5, zorder=4)
    for i, s in g_src.iterrows():
        ax.text(s.geometry.x, s.geometry.y, f" {src_df.loc[i,'name']}", fontsize=8, color="cyan")
    # Observations
    g_obs.plot(ax=ax, color="white", markersize=35, edgecolor="black", linewidth=0.5, zorder=5)
    for i, r in g_obs.iterrows():
        ax.text(r.geometry.x, r.geometry.y, f" {r['site']} ({r['pm25']:.1f})", fontsize=7, color="white")
    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
    ax.set_title(f"{label}\n{stamp} — {title_wind}\nStability {STABILITY}")
    cbar = plt.colorbar(sc, ax=ax, pad=0.02, shrink=0.8)
    cbar.set_label("PM₂.₅ (µg m⁻³)")
    out_png = os.path.join(OUT_DIR, fname)
    plt.tight_layout(); plt.savefig(out_png, bbox_inches="tight"); plt.close(fig)
    print("Saved:", out_png)

plot_surface(C_ugm3, "Forward plume (using configured Q)", f"gplume_forward_{stamp.replace(':','-')}.png")

if C_inv_ugm3 is not None:
    plot_surface(C_inv_ugm3, "Inverse fit plume (Q estimated from stations)", f"gplume_inverse_{stamp.replace(':','-')}.png")
    if Q_est is not None:
        for name, q in zip(src_df["name"], Q_est):
            print(f"Estimated Q for {name}: {q:.4f} g/s")

# ---------------- OPTIONAL: use PyPI 'gplume' package (forward only) ------
if USE_PYPI_GPLUME:
    try:
        # Example stub: call into gplume package (see PyPI README for details)
        from gplume import gpm  # module
        print("[info] gplume module imported; adapt calls to gpm.gplume()/forward_atmospheric_dispersion() as needed.")
        # (API details: https://pypi.org/project/gplume/ ; https://github.com/VaibhavVasdev/Gaussian-Plume_Model )
    except Exception as e:
        print("[warn] gplume not available or API changed:", e)
# --------------------------------------------------------------------------


C:\Users\HP\AppData\Local\Temp\ipykernel_20744\4210941097.py:190: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  mask_in = g_grid.within(g_pasig.unary_union)


Saved: C:\Users\HP\Desktop\SpatialCARE\Hourly\HourlyOutputs\gplume\gplume_forward_2025-01-01 00-00.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Hourly\HourlyOutputs\gplume\gplume_inverse_2025-01-01 00-00.png
Estimated Q for SrcA: 509801844846855805052631829680133777653760.0000 g/s
